# Benchmarking spike sorters


To benchmark the spike sorting algorithms, one of the easiest option is to design artifical dataset, in order to know exactly what is created, and to have a proper "ground truth" to compare with. 

To do so, MEArec is a python package that can help you to generate such artificial datasets. Basically, given some templates and a probe layout, the software will generate traces that can later be used for benchmarking the sorting algorithms. In this notebook, we'll try to get a quick overview of the features allowed by MEArec such that you can test the limits of various spike sorters. 

The comparison between different sorters can be tedious, since every one of them has a different file format. However, spikeinterface can act as a universal wrapper allowing you to launch and read the results of the sorters. Moreover, spikeinterface comes with numerous analysis functions that will allow you to quickly assess the quality of a spike sorting, and compute quality metrics with respect to the Ground Truth that have been generated


The only file needed here will be "templates_Neuronexus-32_100.h5", that we will use to generate the artificial recordings.

In [2]:
%matplotlib inline

In [3]:
from pathlib import Path
import os

import numpy as np
import matplotlib.pyplot as plt

import MEArec as mr

import neo
import quantities as pq

import spikeinterface.full as si

/usr/lib/python3/dist-packages/datalad/cmd.py:377: RuntimeWarning: coroutine 'run_async_cmd' was never awaited
  new_loop = True
/home/pierre/github/spikeinterface/spikeinterface/core/datasets.py:12: RuntimeWarning: coroutine 'run_async_cmd' was never awaited
  HAVE_DATALAD = False


## Step 1 : generation of the recordings

Here is a small code that will generate a recording given an already generated "templates" file. We want to generate an artificial recording with 10 cells, randomly aranged on a Neuronexus layout. The sampling rate will be 30kHe, and the firing rate of the neurons will be arbitrarily set to 5Hz.



In [4]:
basedir = Path('.')
template_filename = basedir / 'templates_Neuronexus-32_100.h5'


duration = 30*60 # duration (in s) of the recording
n_cell = 10 # number of cells we want to create
probe = 'Neuronexus-32' # probe layout
recording_filename = basedir / f'recordings_collision_{n_cell}cells_{probe}_{duration:0.0f}s.h5'

In [5]:
# some parameters should be provided
fs = 30000. # sampling rate (in Hz)
spikerate = 5. # firing rate of the cells (in Hz)


First, we need to generate the spike trains that we want to inject into our traces. The easiest thing is to generate Poisson spike trains. To do so, we will rely on MEARec to generate an artificial recording, given our pre-generated templates and a desired firing rate for the neurons, acting as Poisson sources (default in MEARec). The amount of noise in the recording can be controlled via the parameter noise_level. In order to control exactly the data that will be generated, we will also force the seed of the templates (to always use the same) and of the spiketrains (to always generate the same spike trains)

In [6]:
rec_params = mr.get_default_recordings_params()
rec_params['recordings']['fs'] = fs
rec_params['recordings']['sync_rate'] = None
rec_params['recordings']['sync_jitter'] = 5
rec_params['recordings']['noise_level'] = 5
rec_params['recordings']['filter'] = False
rec_params['recordings']['chunk_duration'] = 10.

rec_params['spiketrains']['f_exc'] = spikerate
rec_params['spiketrains']['seed'] = 42
rec_params['spiketrains']['duration'] = duration
rec_params['spiketrains']['n_exc'] = n_cell
rec_params['spiketrains']['n_inh'] = 0

rec_params['templates']['n_overlap_pairs'] = None
rec_params['templates']['min_dist'] = 0
rec_params['templates']['seed'] = 42

recgen = mr.gen_recordings(params=rec_params, 
            templates=template_filename, verbose=True,
            n_jobs=1, tmp_mode='memmap')



Spiketrains seed:  40
Noise Level  5
Templates selection seed:  8075
Selecting cells
Padding template edges
Elapsed pad time: 0.13539767265319824
Elapsed resample time: 0.00801229476928711
Creating time jittering
Elapsed jitter time: 0.26135802268981934
Computing spike train SNR
Adding spiketrain annotations
Convolution seed:  5764
Electrode modulaton
Adding noise
Noise seed:  7382
Elapsed time:  146.46042824399956


In [12]:
# Then we can save our recording
mr.save_recording_generator(recgen, filename=recording_filename)

## Step 2 : opening and ploting the signals and the spikes from our ground truth

Now we want to use spikeinterface to load/play/visualize the traces that have just been generated. For example, you should use the MEArecRecordingExtractor/MEArecSortingExtractor objects to load the MEArec recording (both the traces and the spike times). The recording should be called rec_gt and the sorting sorting_gt

In [ ]:
#### Loading the recording and the sorting

Once you have loaded the data, you can plot the timeseries with the widgets provided by spikeinterface, such as for example plot_timeseries()

In [ ]:
#### Plot the time series between 5 and 6 seconds

Similarly, you can also, from the same object, plot the raster plot since all the spikes were included in the MEArecSortingExtractor. To do so, use the plot_rasters() function

In [ ]:
#### Plot all the spikes between 5 and 6 seconds

And to check that your neurons are indeed behaving as Poisson sources, could you find a widget to plot the auto/crosscorrelograms for your population? What do you see, and what do you expect?

In [ ]:
#### Plot the correlograms for all units, and describe what you should see

## Step 3 : running several sorters on our recording

With spikeinterface, launching a spike sorting algorithm is easy. You simply need to do use the run_sorter command. Assuming we want to save every sorting into a specific folder, with a name dedicated to every sorter, we can simply do

In [17]:
sorting_spykincircus = si.run_sorter('spykingcircus', rec, output_folder=basedir / 'spykingcircus')
sorting_tridesclous = si.run_sorter('tridesclous', rec, output_folder=basedir / 'tridesclous')

# Note that the sorter list can includes 'kilosort', 'herdingspikes', ... See spikeinterface wrapper for more info

RUNNING SHELL SCRIPT: spykingcircus/run_spykingcircus.sh
h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


Once a given sorter has been launched, you can load its results easily. Can you load the results, using the read_sorter_folder() command, and plot a raster plot similar to the one you plotted for the ground truth units, just before? Do they match?

You just noticed how easy it is to run several sorters on a given recording. But what about comparisons? Now we would like to be able to quantify how good these sorters are, with respect to the ground truth units that have been created. 

## Step 4 : running comparison and ploting agreement matrix

Once we have a given spike sorting, we can always compare it to its ground-truth, via the comparison object offered by spikeinterface. To do so, you can use the function compare_sorter_to_ground_truth() that will create a GroundTruthComparison object that you will call comp, comparing the ground truth data with a sorting you just obtained, with the sorter of your choice

In [ ]:
#### Create a comp object to compare a sorter and a ground truth

Such a comparison object can immediatly tell us how many units are found, what are the errors rates, and much more. For example, try to understand how to visualize the agreement_scores between all the units (ground truth vs find ones). What are these scores? Please have a to the classical ways of measuring how "good" a sorter is. What are the mathematical definition of accuracy, precision, recall? What about F1 score? 
To know more about this comparison module, have a look to the documentation https://spikeinterface.readthedocs.io/en/latest/module_comparison.html

In [ ]:
#### Try to see ho to obtain agreement_scores, or quality metrics from the comparison


Using the plot_agreement_matrix() command, you can have a quick look to the accuracy of your spike sorter, by comparing how good ground truth units are "found" by the sorter, and if these matches are unique. Again, have a look to the documentation to understand what this agreement matrix really means. What does that means if it is not squared, but rectangular? Larger in the horizontal or the vertical direction? What if the diagonal is not filled with 1, and/or if a line have several high values in it?

In [ ]:
#### Plot the agreeement matrix between a sorter and the ground truth

## Step 5 : using the study object and plotting performances

What if we have several spike sorting algorithms, and want to quickly compare each of them, in order to get a sense of the pros and cons of every sorters? This can also be done easily in spikeinterface via the study object. The study can compare several sorters not only on a single, but on multiple recordings at once. In order to probe its potentiel, let's generate a second recording via MEAreac, for example with a larger noise level, but same spike trains. Note that if you prefer, you could also change the firing rate, by generating new spike trains. Depends what you want to probe. 

In [23]:
recording_filename_2 = basedir / f'recordings_collision_{n_cell}cells_{probe}_{duration:0.0f}s_noisy.h5'

rec_params['recordings']['noise_level'] = 8

recgen = mr.gen_recordings(params=rec_params,  
            templates=template_filename, verbose=True,
            n_jobs=1, tmp_mode='memmap')

# Then we can save our recording
mr.save_recording_generator(recgen, filename=recording_filename_2)

Noise Level  8
Templates selection seed:  4893
Selecting cells
Padding template edges
Elapsed pad time: 0.1422104835510254
Elapsed resample time: 0.01348733901977539
Creating time jittering
Elapsed jitter time: 0.25230884552001953
Computing spike train SNR
Adding spiketrain annotations
Convolution seed:  3538
Electrode modulaton
Adding noise
Noise seed:  5793
Elapsed time:  145.80792285200005
Deleted /tmp/tmp68mxnjzt


Note that by default, the seed used to select n_cell templates out of the pre-generated dictionary is not the same, thus the sorting will be different. But this could have been controled via MEArec, in order to change only the noise level

Here, you should try to load the new recording/sorting you just created, as you did for the first one. The recording could be named rec_2, and the sorting sorting_gt_2

In [24]:
#### Loading the second recording and the associated spikes (ground truth)

Using the GroundTruthStudy object, create a study in a folder of your choice (could simply be 'study') such that you will compare sorters on the two differents recordings you have created. Note that the study object accepts a dict of sortings as inputs

In [ ]:
#Now we can create the dictionary of all recordings used by the study
gt_dict = {'rec0' : (rec, sorting_gt), 'rec1' : (rec_2, sorting_gt_2)}

#### Create a study object that will compare the two recordings

The study object is gathering all the recordings in the specified folder, this is why we need to specify chunk_memory and number of jobs used to copy evertyhing. Now that we have the study created, we can easily launch it on several sorters, either for all recordings or for a subset

In [1]:
#### Load the study with the run_sorters() command, on the sorters of your choice

Now that the study has been ran, we need to perform the comparisons between all sorters and the ground truths. Note that if we have an exhaustive ground truth, i.e. if we have a full description of our artificial recordings with the sorting (which is the case here), then it must be specified. This will enhance the quality metrics and provide more information

In [28]:
study.run_comparisons(exhaustive_gt=True)

You can plot the performances over all recordings, and for all sorters, using the plot_gt_study_performances command. What are the pros and cons of the sorters?

In [2]:
#### Comparing the sorters with the plot_gt_study_performences()

In [3]:
#### Comparing the sorters with the plot_gt_study_performences_averages()